dict형은 애플리케이션에서 널리 사용될 뿐만 아니라 파이썬 구현의 핵심 부분이기도 하다.  
모듈 네임스페이스, 클래스 및 인스턴스 속성, 함수의 키워드 인수 등 핵심 부분에 딕셔너리가 사용되고 있다.  
내장 함수들은 `__builtins__.__dict__`에 들어 있다.  
중요한 역할을 맡고 있으므로 파이썬 dict 클래스는 상당히 최적화되어 있다.  
파이썬의 고성능 딕셔너리 뒤에는 **해시 테이블**이라는 엔진이 있다.  
집합도 해시 테이블을 이용해서 구현, 해시 테이블 장동 방식을 알아야 딕셔너리와 집합을 최대로 활용할 수 있다.  
이 장에서는 다음과 같은 내용을 설명한다.
- 공통적으로 사용되는 딕셔너리 메서드
- 없는 키에 대한 특별 처리
- 표준 라이브러리에서 제공하는 다양한 딕셔너리 클래스
- set과 frozenset형
- 해시 테이블의 작동 방식
- 해시 테이블의 의미(키 자료형 제한, 에측할 수 없는 순서 등)

## 3.1 일반적인 매핑형
collections.abc 모듈은 dict 및 이와 유사한 자료형의 인터페이스를 정의하기 위해 Mapping 및 MutableMapping 추상 베이스 클래스(ABC)를 제공.  

In [8]:
my_dict = {}
import collections
isinstance(my_dict, collections.abc.Mapping)

True

함수 인수가 dict형인지 검사하는 것보다 `isinstance()`함수를 사용하는 것이 좋다. 다른 매핑형이 사용될 수도 있기 때문.  
표준 라이브러리에서 제공하는 매핑형은 모두 dict를 이용해서 구현하므로, 키가 **해시 가능**해야 한다는 제한이 있다.(값은 노상관)   
**해시 가능**이란 말의 의미는?  
> 수명 주기 동안 변하지 않는 해시값을 갖고 있고(`__hash__()`메서드 필요) 다른 객체와 비교할 수 있으면(`__eq__()`메서드 필요) 해시가능  
> 동일하다고 판단되는 객체는 반드시 해시값이 동일해야 한다.

원자적 불변형(str, byte, 수치형)은 모두 해시 가능하다. frozenset은 언제나 해시 가능하다.  
모든 요소가 해시 가능하도록 정의되어 있기 때문.  
튜플은 들어 있는 항목들이 모두 해시 가능해야 해시 가능하다.  

In [3]:
tt = (1, 2, (30, 40))
print(hash(tt))
t1 = (1, 2, [30, 40])
print(hash(t1))

8027212646858338501


TypeError: unhashable type: 'list'

In [4]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

-4118419923444501110

In [5]:
help(frozenset)

Help on class frozenset in module builtins:

class frozenset(object)
 |  frozenset() -> empty frozenset object
 |  frozenset(iterable) -> frozenset object
 |  
 |  Build an immutable unordered collection of unique elements.
 |  
 |  Methods defined here:
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __contains__(...)
 |      x.__contains__(y) <==> y in x.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __len__(self, /)
 |      Return len(self).
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __new__(*arg

In [6]:
ts = (1, 2, {30, 40})
hash(ts)

TypeError: unhashable type: 'set'

frozenset: 변경 불가능한 set 자료구조. 나머지 연산은 동일.. 그냥 set과 달리 변경 불가능하므로 hash 가능하다.

사용자 정의 자료형은 기본적으로 해시 가능하다. 기본적으로 객체의 해시값은 id()를 이용해서 구하므로 모든 객체가 서로 다르기 때문.  
객체가 자신의 내부 상태를 평가해서 `__eq__()`메서드를 직접 구현하는 경우에는 해시값 계산에 사용되는 속성이 모두 불변형일 때만 해시 가능.  

딕셔너리를 구현하는 다양한 방법

In [7]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

## 3.2 지능형 딕셔너리
파이썬 2.7부터 comprehension 표현식이 딕셔너리와 set에도 적용.  
지능형 딕셔너리는 모든 반복형 객체에서 키-값 쌍을 생성함으로써 딕셔너리 객체를 만들 수 있다.

In [9]:
DIAL_CODES = [                # 1
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'),
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan'),
]
country_code = {country: code for code, country in DIAL_CODES}    # 2
print(country_code)
{code: country.upper() for country, code in country_code.items() if code < 66}    # 3

{'United States': 1, 'Japan': 81, 'India': 91, 'Brazil': 55, 'Indonesia': 62, 'Bangladesh': 880, 'Pakistan': 92, 'China': 86, 'Russia': 7, 'Nigeria': 234}


{1: 'UNITED STATES', 7: 'RUSSIA', 55: 'BRAZIL', 62: 'INDONESIA'}

1. dict 생성자에 키-값 쌍의 리스트를 바로 사용할 수 있다.
2. 쌍을 뒤바꿔서 country는 키, code는 값이 된다.
3. 쌍을 한 번 더 뒤바꿔서 값을 대문자로 바꾸고, code가 66보다 작은 항목만 걸러낸다.

## 3.3 공통적인 매핑 메서드
매핑이 제공하는 기본 API는 아주 많다. dict의 변형 중 가장 널리 사용되는 defaultdict와 OrderedDict 클래스가 구현하는 메서드를 보자.  
```
메서드                   dict     defaultdict     OrderedDict   설명
-------------------------------------------------------------------------------------------------
d.clear()                  o          o               o       모든 항목을 제거한다.
d.__contains__(k)          o          o               o       k in d
d.copy()                   o          o               o       얕게 복사한다.
d.__copy__()                          o                       copy.copy()를 지원한다.
d.default_factory                     o                       빠진 값을 설정하기 위해 `__missing__()`메서드에 호출되는 콜러블  
d.__delitem__(k)           o          o               o       del d[k] -- 키가 k인 항목을 제거한다.  
d.fromkeys(it, [initial])  o          o               o       선택적인 초깃값(기본 None)을 받아, 반복 가능한 객체의 키들을 이용해서 
_                                                             새로 매핑한다.  
d.get(k, [default])        o          o               o       k키를 가진 항목을 반환한다. 없으면 default나 None 반환
d.__getitem__(k)           o          o               o       d[k] -- k 키를 가진 항목을 반환
d.items()                  o          o               o       (키, 값)쌍으로 구성된 항목들의 뷰를 가져온다.
d.__iter__()               o          o               o       키에 대한 반복자를 가져온다.
d.keys()                   o          o               o       키에 대한 '뷰'를 가져온다.
d.__len__()                o          o               o       len(d) -- 항목 수를 반환한다.
d.__missing__(k)                      o                       __getitem__()이 k 키를 찾을 수 없을 때 호출
d.move_to_end(k, [last])                              o       앞이나 뒤에서 k개의 항목을 이동(last 기본값을 True)
d.pop(k, [default])        o          o               o       k 키 항목을 제거하고 반환한다. 없으면 default나 None반환
d.popitem()                o          o               o       처음이나 마지막 (키, 값)항목을 제거하고 반환한다.
d.__reversed__()                                      o       키에 대한 역순 반복자를 가져온다.
d.setdefault(k, [default]) o          o               o       k in d가 참이면 d[k]를 반환, 아니면 d[k]=default로 설정 후 값 반환.
d.__setitem__(k, v)        o          o               o       d[k] = v -- k 키를 가진 항목의 값을 v로 설정한다.
d.update(m, [**kargs])     o          o               o       (키, 값)쌍의 매핑이나 반복형 객체에서 가져온 항목들로 d를 갱신
d.values()                 o          o               o       값들에 대한 뷰를 가져온다.
```

update 메서드가 첫 번째 인수 m을 다루는 방식은 덕 타이핑의 대표적 사례다.  
먼저 m이 keys() 메서드를 갖고 있는지 확인한 후, 만약 메서드를 갖고 있으면 매핑이라고 간주한다.  
keys() 메서드가 없으면, update() 메서드는 m의 항목들이 (키, 값) 쌍으로 되어 있다고 간주하고 m을 반복한다.  
대부분의 파이썬 매핑은 update() 메서드와 같은 논리를 내부적으로 구현한다.  
따라서 매핑은 다른 매핑으로 초기화하거나, (키, 값)쌍을 생성할 수 있는 반복형 객체로 초기화할 수 있다.  
매핑의 setdefault() 메서드가 늘 필요한 것은 아니지만 이 메서드가 필요할 때는 똑같은 키를 여러 번 조회하지 않게 해줌으로써 속도를 엄청나게 향상시킨다. 

### 3.3.1 존재하지 않는 키를 setdefault()로 처리하기
**조기 실패(fail-fast)** 철학에 따라, 존재하지 않는 키 k로 d[k]를 접근하면 dict는 오류 발생.  
KeyError를 처리하는 것보다 기본값을 사용하는 것이 더 편리한 경우에는 d[k] 대신 d.get(k, default)를 사용.  
그렇지만 발견한 값을 갱신할 때, 해당 객체가 가변 객체면 `__getitem__()`이나 get() 메서드는 보기 어색하고 효율성도 떨어진다.  
다음은 존재하지 않는 키를 처리할 때 dict.get()이 좋지 않은 사례.

In [ ]:
"""단어가 나타나는 위치를 가리키는 인덱스를 만든다."""
import sys
import re

WORD_RE = re.compile(r'\w+')

index = {}

with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            occurrences = index.get(word, [])  # 1
            occurrences.append(location)       # 2
            index[word] = occurrences          # 3
            
# 알파벳순으로 출력
for word in sorted(idnex, key=str.upper):      # 4
    print(word, index[word])

1. 단어(word)에 대한 occurrences 리스트를 가져오거나, 단어가 없으면 빈 배열([])을 가져온다.
2. 새로 만든 location을 occurrences에 추가한다.
3. 변경된 occurrences를 index 딕셔너리에 넣는다. 그러면 index를 한 번 더 검색한다.
4. sorted() 함수의 key 인수 안에서 str.upper()를 호출하지 않고, 단지 str.upper() 함수에 대한 참조를 전달해서 sorted() 함수가 이 함수를 이용해서 정렬할 단어를 정규화하게 만든다.
    - 이것은 메서드를 일급 함수로 사용하는 하나의 예.

위의 1, 2, 3번 세 줄 코드를 dict.setdefault()를 사용하여 한 줄로 바꿀 수 있다.

In [ ]:
index.setdefault(word, []).append(location)

setdefault()가 값을 반환하므로 한 번 더 검색할 필요 없이 갱신할 수 있다.

```python
mydict.setdefault(key, []).append(new_value)
```
위 코드는 
```python
if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value)
```
와 같다.  
밑의 코드는 키를 두 번 검색하고 단어가 없는 경우 세 번 검색하지만 setdefault() 코드는 단 한 번만 검색해서 모든 과정을 수행한다.  

## 3.4 융통성 있게 키를 조회하는 매핑
검색할 때 키가 존재하지 않으면 어떤 특별한 값을 반환하는 매핑이 있으면 편리한 때가 종종 있다.  
평범한 dict 대신 defaultdict를 사용할 수 있고, dict 등의 매핑형을 상속해서 `__mising__()` 메서드를 추가하는 방법이 있다.  

### 3.4.1 defaultdict: 존재하지 않는 키에 대한 또 다른 처리
defaultdict는 존재하지 않는 키로 검색할 때 요청에 따라 항목을 생성하도록 되어있다.  
defaultdict 객체를 생성할 때 존재하지 않는 키 인수로 `__getitem()` 메서드를 호출할 때마다 기본값을 생성하기 위해 사용되는 콜러블 제공.  
`dd = defaultdict(list)` 코드로 기본 defaultdict 객체를 생성한 후, dd에 존재하지 않는 키인 'new-key'로 dd['new-key'] 표현식을 실행하면,  
1. 리스트를 새로 생성하기 위해 list()를 호출한다.
2. 'new-key'를 키로 사용해서 새로운 리스트를 dd에 삽입한다.
3. 리스트에 대한 참조를 반환한다.  
  
기본값을 생성하는 콜러블은 defaut_factory라는 객체 속성에 저장된다.  
이 default_factory가 설정되어 있지 않으면, 키가 없을 때 볼 수 있는 KeyError 발생.

defaultdict의 default_factory는 `__getitem__()`호출에 대한 기본값을 제공하기 위해 호출되며, 다른 메서드를 통해서는 호출되지 않는다.  
예를 들어 dd가 defaultdict형이며, k가 존재하지 않는 키면, dd[k]는 default_factory를 호출해서 기본값을 생성하지만,  
dd.get(k)는 단지 None을 반환.  

실제 defaultdict가 default_factory를 호출하게 만드는 메커니즘은 `__missing__()`특수 메서드에 의존하며, 표준 매핑형은 모두 이 기능 지원.  

### 3.4.2 `__mising__()`메서드

매핑형은 이 메서드를 이용해서 존재하지 않는 키를 처리한다.  
이 특수 메서드는 기본 클래스인 dict에는 정의되어 있지 않지만, dict는 이 메서드를 알고 있다.  
따라서 dict 클래스를 상속하고 `__missing__()`메서드를 정의하면, `dict.__getitem__()`표준 메서드가 키를 발견할 수 없을 때 KeyError를 발생시키지 않고 `__missing__()`메서드를 호출한다.  
`__missing__()`메서드는 d[k] 연산자를 사용하는 경우 등 `__getitem__()` 메서드를 사용할 때만 호출된다.  
in 연산자를 구현하는 get()이나 `__contains__()`메서드 등 키를 검색하는 다른 메서드에는 `__missing__()`메서드가 영향을 미치지 않는다.  

> **NOTE_** 사용자 정의 매핑형을 만들 때는 dict보다 collectionsUserDict 클래스를 상속하는 것이 더 낫다.  
여기서는 `dict.__getitem__()`내장 메서드가 `__missing__()`메서드를 지원한다는 것을 보여주기 위해 dict 클래스 상속.

In [19]:
# 조회할 때 키를 문자열로 변환하는 StrKeyDict0
class StrKeyDict0(dict):                                         # 1
     
    def __missing__(self, key):
        if isinstance(key, str):                                 # 2
            raise KeyError(key)
        return self[str(key)]                                    # 3
    
    def get(self, key, default=None):
        try:
            return self[key]                                     # 4
        except KeyError:
            return default                                       # 5
        
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()     # 6

1. StrKeyDict0이 dict를 상속한다.
2. 키가 문자열인지 확인한다. 키가 문자열이고 존재하지 않으면 KeyError가 발생한다.
3. 키에서 문자열을 만들고 조회한다.
4. get()메서드는 self[ky] 표기법을 이용해서 `__getitem__()`메서드에 위임한다. 이렇게 함으로써 `__missing__()`메서드 작동 기회를 준다.  
5. KeyError가 발생하면 `__missing__()`메서드가 이미 실패한 것이므로 default를 반환한다.
6. .수정하지 않은 (문자열이 아닐 수 있는) 키를 검색하고 나서, 키에서 만든 문자열로 검색한다.

일관성을 위해 `__contains__`메서드까지 구현.  
k in my_dict 스타일로 조회하면 재귀적으로 호출됨.. 따라서 key in self.keys()와 같이 명시적으로 키 조회.  
파이썬 2에서는 dict.keys()가 리스트임에 주의..

In [20]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
print(d['2'])
print(d[4])
print(d[1])

two
four


KeyError: '1'

In [21]:
print(d.get('2'))
print(d.get(4))
print(d.get(1, 'N/A'))

two
four
N/A


## 3.5 그 외 매핑형
collections 모듈에서 제공하는 여러 매핑형.  
1. collections.OrderedDict
    - 키를 삽입한 순서대로 유지함으로써 항목을 반복하는 순서를 예측할 수 있다.
    - popitem()메서드는 기본적으로 최근에 삽입한 항목을 꺼내지만 popitem(last=True)형태로 호출하면 처음 삽입한 항목을 꺼낸다.  
2. collections.ChainMap
    - 매핑들의 목록을 담고 있으며 한꺼번에 모두 검색할 수 있다. 
    - 각 매핑을 차례대로 검색하고, 그중 하나에서라도 키가 검색되면 성공한다.  

In [25]:
import builtins
from collections import ChainMap
from pprint import pprint

pylookup = ChainMap(locals(), globals(), vars(builtins))
pprint(pylookup)

ChainMap({'ChainMap': <class 'collections.ChainMap'>,
          'DIAL_CODES': [(86, 'China'),
                         (91, 'India'),
                         (1, 'United States'),
                         (62, 'Indonesia'),
                         (55, 'Brazil'),
                         (92, 'Pakistan'),
                         (880, 'Bangladesh'),
                         (234, 'Nigeria'),
                         (7, 'Russia'),
                         (81, 'Japan')],
          'In': ['',
                 'my_dict = {}\n'
                 'import collectios\n'
                 'isinstance(my_dict, collections.abc.Mapping)',
                 'my_dict = {}\n'
                 'import collections\n'
                 'isinstance(my_dict, collections.abc.Mapping)',
                 'tt = (1, 2, (30, 40))\n'
                 'print(hash(tt))\n'
                 't1 = (1, 2, [30, 40])\n'
                 'print(hash(t1))',
                 'tf = (1, 2, frozenset([30, 40]))\nhash(tf)',
 

                  'import builtins\n'
                  'from collections import ChainMap\n'
                  '\n'
                  'pylookup = ChainMap(locals(), globals(), vars(builtins))',
                  'import builtins\n'
                  'from collections import ChainMap\n'
                  '\n'
                  'pylookup = ChainMap(locals(), globals(), vars(builtins))\n'
                  'pylookup',
                  'import builtins\n'
                  'from collections import ChainMap\n'
                  'from pprint import pprint\n'
                  '\n'
                  'pylookup = ChainMap(locals(), globals(), vars(builtins))\n'
                  'pprint(lookup)',
                  'import builtins\n'
                  'from collections import ChainMap\n'
                  'from pprint import pprint\n'
                  '\n'
                  'pylookup = ChainMap(locals(), globals(), vars(builtins))\n'
                  'pprint(pylookup)'],
          '_ii': 'im

3. collections.Counter
    - 모든 키에 정수형 카운터를 갖고 있는 매핑. 기존 키를 갱신하면 카운터가 늘어난다.  
    - 해시 가능한 객체(키)나 한 항목이 여러 번 들어갈 수 잇는 다중 집합(multiset)에서 객체의 수를 세기 위해 사용할 수 있다.  
    - Counter 클래스는 합계를 구하기 위한 +와 -연산자를 구현하며, n개의 가장 널리 사용된 항목과 그들의 카운터로 구성된 튜플의 리스트를 반환하는 most_common([n])등의 메서드를 제공한다.

In [27]:
from collections import Counter
ct = Counter('abracadabra')
print(ct)
ct.update('aaaaazzz')
print(ct)
print(ct.most_common(2))

Counter({'a': 5, 'r': 2, 'b': 2, 'c': 1, 'd': 1})
Counter({'a': 10, 'z': 3, 'b': 2, 'r': 2, 'c': 1, 'd': 1})
[('a', 10), ('z', 3)]


4. collections.UserDict
    - 표준 dict처럼 작동하는 매핑을 순수 파이썬으로 구현한 클래스
    - 상속해서 사용하도록 설계되어 있다.


## 3.6 UserDict 상속하기
dict보다는 UserDict를 상속해서 매핑형을 만드는 것이 쉽다.  
매핑에 추가한 키를 문자열로 저장하기 위해 StrKeyDict0을 확장했던 것처럼 UserDict의 값을 평가할 수 있다.   
내장형에서 아무런 문제없이 상속할 수 있는 메서드들을 오버라이드해야 하는 구현의 특이성 때문에 dict보다 UserDict를 상속하는 것이 낫다.  
(내장 자료형의 상속은 까다롭다!)  
UserDict는 dict를 상속하지 않고 내부에 실제 항목을 담고 있는 data라고 하는 dict 객체를 갖고 있다.  
이렇게 구현함으로써 `__setitem__()`등의 특수 메서드를 구현할 때 발생하는 원치 않는 재귀적 호출을 피할 수 있으며,  
`__contains__()`메서드를 간단히 구현할 수 있다.  
덕분에 StrKeyDict0 예제보다 간단하지만 더 많은 일을 하도록 구현할 수 있다.  

In [28]:
from collections import UserDict

class StrKeyDict(UserDict):                # 1
    
    def __mising__(self, key):             # 2
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data       # 3
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item         # 4

1. StrKeyDict는 UserDict를 상속한다.
2. `__missing__()` 메서드는 StrKeyDict0 예제와 같다.
3. `__contains__()`메서드는 더 간단하다. 저장된 키가 모두 str형이므로 self.data에서 바로 조회할 수 있다.
4. `__setitem__()` 메서드는 모든 키를 str형으로 변환하므로, 연산을 self.data에 위임할 때 더 간단히 작성할 수 있다. 

UserDict 클래스가 MutableMapping을 상속하므로 StrKeyDict는 결국 UserDict, MutableMapping, 또는 Mapping을 상속하여 모든 매핑의 기능을 가짐.  
Mapping은 추상 베이스 클래스(ABC)임에도 불구하고 유용한 구상(concrete, 구체적) 메서드를 다수 제공한다.  
1. MutableMapping.update()
    - 직접 호출할 수도 있지만, 다른 매핑이나 (키, 값)쌍의 반복형 및 키워드 인수에서 객체를 로딩하기 위해 `__init__()`에서 사용될 수도 있다. 
    - 항목을 추가하기 위해 'self[k] = v' 구문을 사용하므로 결국 서브클래스에서 구현한 `__setitem__()`메서드를 호출하게 된다.
2. Mapping.get()
    - StrKeyDict0.get()과 완전히 동일하게 구현된 Mapping.get()을 상속받음..

불변 시퀀스형이 여러 종류 있는데 그렇다면 불변 딕셔너리는 어떨까?  
표준 라이브러리에는 들어있지 않지만 추가할 수는 있다.

## 3.7 불변 매핑
표준 라이브러리에서 제공하는 매핑형은 모두 가변형이지만, 사용자가 실수로 매핑을 변경하지 못하도록 보장하고 싶은 경우.  
파이썬 3.3 이후 types 모듈은 MappingProxyType이라는 래퍼 클래스를 제공해서, 원래 매핑의 동적인 뷰를 제공하지만 읽기 전용의 mappingproxy 객체를 반환한다.  
따라서 원래 매핑을 변경하면 mappingproxy에 반영되지만, mappingproxy를 직접 변경할 수는 없다.  

In [31]:
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
print(d_proxy)
print(d_proxy[1])              # 1
d_proxy[2] = 'x'               # 2

{1: 'A'}
A


TypeError: 'mappingproxy' object does not support item assignment

In [32]:
d[2] = 'B'
print(d_proxy)              # 3
print(d_proxy[2])

{1: 'A', 2: 'B'}
B


1. d에 들어 있는 항목은 d_proxy를 통해서 볼 수 있다.
2. d_proxy를 변경할 수는 없다.
3. 동적인 d_proxy는 d에 대한 변경을 바로 반영한다!!!

## 3.8 집합 이론

set 형과 set의 불변형 버전인 frozenset은 파이썬 2.3에 모듈로 처음 등장했으며 파이썬 2.6에서 내장형으로 승격되었다.  
집합은 고유한 객체의 모음으로서, 기본적으로 중복 항목을 제거한다.

In [1]:
l = ['spam', 'spam', 'eggs', 'spam']
print(set(l))
print(list(set(l)))

{'spam', 'eggs'}
['spam', 'eggs']


집합 요소는 반드시 해시할 수 있어야 한다. set은 해시 가능하지 않지만 frozenset은 해시 가능하므로, frozenset이 set에 들어갈 수 있다.  

고유함을 보장하는 것 외에 집합형은 중위 연산자를 이용해서 기본적인 집합 연산을 구현한다.  
두 개의 집합 a, b가 있을 때, a | b는 합집합, a & b는 교집합, a - b는 차집합을 계산한다.  
이메일 주소가 들어 있는 큰 집합(haystack)과 몇 가지 이메일 주소가 들어 있는 작은 집합(needles)이 있을때  
needles에 들어 있는 이메일 중 몇 개가 haystack 안에도 들어 있는지 알고 싶다면 교집합(&)연산자를 통해 간단히 알 수 있다.  

In [ ]:
found = len(needles & haystack)

교집합 연산자를 사용하지 않고 반복문을 사용한다면,

In [ ]:
found = 0
for n in needles:
    if n in haystack:
        found += 1

위 코드는 객체가 집합형이 아니더라도 반복 가능형이면 사용할 수 있다.  
하지만 객체가 집합형이 아니더라도 즉석에서 집합으로 만들 수 있다.

In [ ]:
found = len(set(needles) & set(haystack))

# 또 다른 방법
found = len(set(needles).intersection(haystack))

내부 해시 테이블 덕분에 집합 안에 속해 있는지 여부를 아주 빨리 검색할 수 있는 것 외에도  
set과 frozenset 내장 자료형은 새로운 집합을 생성하거나, set의 경우 기존 항목을 변경하는 다양한 연산을 제공한다.

### 3.8.1 집합 리터럴

{1}, {1, 2} 등 집합 리터럴에 대한 구문은 수학적 표기법과 동일하지만, 공집합은 리터럴로 표기할 수 없으므로, 반드시 set()으로 표기한다.  

In [3]:
s = {1}
print(type(s))
print(s)
print(s.pop())
print(s)

<class 'set'>
{1}
1
set()


{1, 2, 3}과 같은 리터럴 집합 구문은 set([1, 2, 3])처럼 생성자를 호출하는 것보다 빠르고 가독성이 좋다.  
생성자를 명시적으로 호출하는 경우에는 파이썬이 생성자를 가져오기 위해 집합명을 검색하고, 리스트를 생성하고, 이 리스트를 생성자에 전달해야 하므로 더 느리다.  
반면 리터럴 집합 구문을 처리한느 경우, 파이썬은 BUILD_SET이라는 특수 바이트코드를 실행한다.  

#### 디스어셈블러 함수인 dis.dis()를 이용해서 두 개의 연산에 대한 바이트코드 살펴보기.

In [ ]:
from dis import dis
dis('{1}')                     # 1

```
  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1     # 2
              4 RETURN_VALUE
```

In [ ]:
dis('set([1])')               # 3

```
  1           0 LOAD_NAME                0 (set)   # 4
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE
```

1. 리터럴 표현식 {1}에 대한 바이트코드로 디스어셈블한다.
2. BUILD_SET 특수 바이트코드가 거의 모든 일을 처리한다.
3. 생성자 set([1])에 대한 바이트코드로 디스어셈블한다.
4. BUILD_SET 대신 LOAD_NAME, BUILD_LIST, CALL_FUNCTION 총 3 개의 연산을 수행한다.

frozenset에 대한 별도의 리터럴 구문은 없으며 언제나 생성자를 호출해서 생성해야 한다.

In [6]:
dis('frozenset([1])')

  1           0 LOAD_NAME                0 (frozenset)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


In [7]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

listcomp형태를 이용해서 집합을 생성할 수도 있다.

### 3.8.2 지능형 집합
지능형 집합(set comprehension; setcomp)은 dictcomp와 함께 파이썬 2.7에 추가되었다.

In [8]:
from unicodedata import name    # 1
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}    # 2

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

1. 문자명을 가져오기 위해 unicodedate에서 name() 함수를 임포트한다.
2. 코드 번호가 32에서 255 사이에 잇는 문자 중 문자명 안에 'SIGN'단어가 들어 있는 문자들의 집합을 생성한다.

In [17]:
for i in range(256):
    try:
        print(name(chr(i)))
    except:
        pass

SPACE
EXCLAMATION MARK
QUOTATION MARK
NUMBER SIGN
DOLLAR SIGN
PERCENT SIGN
AMPERSAND
APOSTROPHE
LEFT PARENTHESIS
RIGHT PARENTHESIS
ASTERISK
PLUS SIGN
COMMA
HYPHEN-MINUS
FULL STOP
SOLIDUS
DIGIT ZERO
DIGIT ONE
DIGIT TWO
DIGIT THREE
DIGIT FOUR
DIGIT FIVE
DIGIT SIX
DIGIT SEVEN
DIGIT EIGHT
DIGIT NINE
COLON
SEMICOLON
LESS-THAN SIGN
EQUALS SIGN
GREATER-THAN SIGN
QUESTION MARK
COMMERCIAL AT
LATIN CAPITAL LETTER A
LATIN CAPITAL LETTER B
LATIN CAPITAL LETTER C
LATIN CAPITAL LETTER D
LATIN CAPITAL LETTER E
LATIN CAPITAL LETTER F
LATIN CAPITAL LETTER G
LATIN CAPITAL LETTER H
LATIN CAPITAL LETTER I
LATIN CAPITAL LETTER J
LATIN CAPITAL LETTER K
LATIN CAPITAL LETTER L
LATIN CAPITAL LETTER M
LATIN CAPITAL LETTER N
LATIN CAPITAL LETTER O
LATIN CAPITAL LETTER P
LATIN CAPITAL LETTER Q
LATIN CAPITAL LETTER R
LATIN CAPITAL LETTER S
LATIN CAPITAL LETTER T
LATIN CAPITAL LETTER U
LATIN CAPITAL LETTER V
LATIN CAPITAL LETTER W
LATIN CAPITAL LETTER X
LATIN CAPITAL LETTER Y
LATIN CAPITAL LETTER Z
LEFT SQUARE BRAC

### 3.8.3 집합 연산
가변형과 불변형 집합에 사용할 수 있는 메서드는 아래 그림과 같다.  
상당수는 연산자를 오버로딩하기 위한 특수 메서드다.  
<img src="https://github.com/ddangu525/Study/blob/master/Python/Fluent%20Python/images/3-2.JPG?raw=true">
clooections.abc 모듈의 MutableSet 및 슈퍼클래스들의 UML 클래스 다이어그램.  
이탤릭체는 추상 클래스나 추상 메서드를 나타내며, 다이어그램을 간단히 표현하기 위해 역순 연산자는 생략.

<img src="https://github.com/ddangu525/Study/blob/master/Python/Fluent%20Python/images/table%203-2.JPG?raw=true">
위 표는 수학 집합 연산자를 파이썬의 해당 연산자나 메서드에 대응해서 설명.  
`&=`와 difference_update()등 일부 연산자와 메서드는 대상 집합을 직접 변경한다.  
이런 연산자들은 대응하는 실제 수학 연산이 없고, frozenset에는 구현되어 있지 않다.

중위 연산자는 양쪽 피연산자가 모두 집합이어야 하지만, 그 외 메서드는 하나 이상의 반복 가능한 인수를 받을 수 있다.  
네 개의 컬렉션 a, b, c, d의 합집합을 구하기 위해 a.union(b, c, d)형태로 호출할 수 있는데, a는 반드시 집합이어야 하지만 b, c, d는 반복 가능한 어떠한 자료형도 사용할 수 있다.

True나 False를 반환하는 비교 연산자 및 메서드는 다음과 같다.
<img src="https://github.com/ddangu525/Study/blob/master/Python/Fluent%20Python/images/table%203-3.JPG?raw=true">

그 외 집합 메서드
<img src="https://github.com/ddangu525/Study/blob/master/Python/Fluent%20Python/images/table%203-4.JPG?raw=true">

지금까지는 집합의 기능에 대해 살펴보았다.  
이제 딕셔너리와 집합이 어떻게 해시 테이블을 이용해서 구현되는지 알아보자.

## 3.9 dict와 set의 내부 구조

해시 테이블을 이용한 구현 방식을 알면, 장점과 단점을 이해하는 데 도움이 된다.  
- 파이썬 dict와 set은 얼마나 효율적인가?
- 왜 순서가 없을까?
- dict의 키와 set 항목에 파이썬의 모든 객체를 사용할 수 없는 이유는 무엇인가?
- dict의 키와 set 항목의 순서가 왜 삽입 순서에 따라 달라지며, 객체 수명주기 동안 이 순서가 바뀔 수 있는 이유는 무엇일까?
- 딕셔너리와 집합을 반복하는 동안 항목을 추가하면 왜 안될까?

### 3.9.1 성능 실험

in 연산자로 검색할 때 dict, set, list의 크기가 성능에 미치는 영향을 확인해보자.  
배밀도 실수 천만 개로 구성된 배열 haystack을 생성하고 나서 haystack에 있는 500개의 실수와 없는 500개의 실수로 구성된 needles 배열을 생성. 

dict.fromkeys()를 사용해서 dict 객체 생성 후 다음 코드의 실행시간 측정.

In [ ]:
found = 0
for n in needls:
    if n in haystack:
        found += 1

집합은 아래 코드로 측정

In [ ]:
found = len(needles & haystack)

haystack의 크기를 1천 ~ 1천만까지 10배씩 증가하면서 성능 측정.  
파이썬 3.4.0, Core i7 랩톱 기준..  
<img src="https://github.com/ddangu525/Study/blob/master/Python/Fluent%20Python/images/table%203-5.JPG?raw=true">

dict, set, list에 대한 시간 측정..
<img src="https://github.com/ddangu525/Study/blob/master/Python/Fluent%20Python/images/table%203-6.JPG?raw=true">

프로그램에서 외부 장치로의 입출력을 수행한다면 dict나 set의 크기에 상관없이 키 검색 시간은 무시할 수 있을 정도로 작다.   
해시 테이블의 내부를 이해하면 키들이 무작위 순서로 불안정하게 정렬되는 이유를 알 수 있다.    

### 3.9.2 딕셔너리 안의 해시 테이블
해시 테이블은 **희소 배열**(Sparse Array; 중간에 빈 항목을 가진 배열)이다.  
해시 테이블 안에 있는 항목을 종종 '버킷(bucket)'이라 한다.  
dict 해시 테이블에는 각 항목별로 버킷이 있고, 버킷에는 키에 대한 참조와 항목의 값에 대한 참조가 들어간다.  
모든 버킷의 크기가 동일하므로 오프셋을 계산해서 각 버킷에 바로 접근할 수 있다.  
파이썬은 **버킷의 1/3 이상을 비워두려고 노력**한다.  
해시 테이블 항목이 많아지면 더 넓은 공간에 복사해서 버킷 공간을 확보한다.  
해시 테이블 안에 항목을 넣을 때, 먼저 항목 키의 해시값을 계산한다.  
해시는 hash() 내장 함수를 이용해서 계산한다.  

#### 해시와 동치성
hash() 내장 함수는 내장 자료형은 직접 처리하고 사용자 정의 자료형의 경우 `__hash__()`메서드를 호출한다.  
두 객체가 동일하면 이 값들의 해시값도 동일해야 한다. 그렇지 않으면 알고리즘이 제대로 작동하지 않는다.  
예를 들어 정수 1과 실수 1의 내부 표현 형태는 다르지만 1 == 1.0이 참이므로 hash(1) == hash(1.0)도 참이 되어야 한다.  
그리고 해시 테이블 인덱스처럼 효율성을 높이려면 해시값이 가능한 한 인덱스 공간에 골고루 퍼져야 한다.  
즉, 이상적으로는 비슷하지만 동일하지 않은 객체들의 해시값은 상당히 달라야 한다.  
1과 1.0의 해시값은 동일하지만, 1.0001, 1.0002, 1.0003의 해시값은 서로 상당히 다르다.

<img src="https://github.com/ddangu525/Study/blob/master/Python/Fluent%20Python/images/example%203-16.JPG?raw=true">

In [18]:
import sys

MAX_BITS = len(format(sys.maxsize, 'b'))
print('%s-bit Python build' % (MAX_BITS + 1))

def hash_diff(o1, o2):
    h1 = '{:>0{}b}'.format(hash(o1), MAX_BITS)
    h2 = '{:>0{}b}'.format(hash(o2), MAX_BITS)
    diff = ''.join('!' if b1 != b2 else ' ' for b1, b2 in zip(h1, h2))
    count = '!= {}'.format(diff.count('!'))
    width = max(len(repr(o1)), len(repr(o2)), 8)
    sep = '-' * (width * 2 + MAX_BITS)
    return '{!r:{width}} {}\n{:{width}} {} {}\n{!r:{width}} {}\n{}'.format(
    		o1, h1, ' ' * width, diff, count, o2, h2, sep, width=width)

if __name__ == '__main__':
    print(hash_diff(1, 1.0))
    print(hash_diff(1.0, 1.0001))
    print(hash_diff(1.0001, 1.0002))
    print(hash_diff(1.0002, 1.0003))

32-bit Python build
1        0000000000000000000000000000001
                                         != 0
1.0      0000000000000000000000000000001
-----------------------------------------------
1.0      0000000000000000000000000000001
          ! !!! ! !! ! !    ! ! !! !!!   != 16
1.0001   0101110101101010000101011011101
-----------------------------------------------
1.0001   0101110101101010000101011011101
         !!!  !!!! !!!!!   !!!!! !!  !   != 20
1.0002   1011101011010100001010110111001
-----------------------------------------------
1.0002   1011101011010100001010110111001
         ! !   ! !!! ! !  !! ! !  ! !!!! != 17
1.0003   0001100000111110010000010010110
-----------------------------------------------


#### 해시 테이블 알고리즘
my_dict[search_key]에서 값을 가져오기 위해 파이썬은 `__hash__(search_key)`를 호출해서 search_key의 해시값을 가져오고,  
해시값의 최하위 비트를 해시 테이블 안의 버킷에 대한 오프셋으로 사용한다.(사용하는 비트 수는 테이블 크기에 따라 달라짐)  
찾아낸 버킷이 비어 있으면 KeyError를 발생시키고, 그렇지 않으면 버킷에 들어 있는 항목인 (found_key : found_value) 쌍을 검사해서  
search_key == found_key인지 검사한다. 이 값이 일치하면 항목을 찾은 것이므로 found_value를 반환한다.  
하지만 search_key와 found_key가 다른 경우에는 **해시 충돌**(hash collision)이 발생한 것이다.  
해시 충돌은 해시 함수가 임의의 객체를 적은 수의 비트로 매핑하기 때문에 발생한다.  
해시 충돌을 해결하기 위해 알고리즘은 해시의 다른 비트들을 가져와서 특정한 방식으로 조작한 후 그 결과를 이용해서 다른 버킷을 조회한다.  
이때 버킷이 비어 있으면 KeyError를 발생시킨다.  
그렇지 않고 키가 일치하면 항목 값을 반환하고, 키가 일치하지 않으면 다시 충돌 해결 프로세스를 반복한다.  

<img src="https://github.com/ddangu525/Study/blob/master/Python/Fluent%20Python/images/Figure%203-3.JPG?raw=true">

항목을 추가하거나 갱신하는 과정도 동일하다.  
다만 빈 버킷을 찾으면 새로운 항목을 추가하고, 동일한 키를 가진 버킷을 찾으면 버킷의 값을 새로운 값으로 갱신한다.

그리고 항목을 추가할 때 해시 테이블에 항목이 너무 많다고 판단되면 더 큰 공간을 가진 새로운 위치에 해시 테이블을 다시 만든다.  
해시 테이블이 커지면 더 많은 해시 비트를 버킷 오프셋으로 사용하며, 더 많은 비트를 사용할수록 충돌률은 낮아진다.  
dict안에 수백만 개의 항목이 있어도 충돌 없이 검색되는 경우가 많고, 한 번 검색할 때마다 발생하는 평균 충돌횟수는 1~2사이다.  
운이 아주 안 좋은 키의 경우에도 몇 번의 충돌만에 원하는 항목을 찾을 수 있다.  
dict의 구현 방식을 알게 되었으니 이 데이터 구조체 및 유도된 다른 모든 구조체의 장단점을 설명할 수 있다.  
파이썬 dict가 왜 그렇게 작동하는지 생각해보자.

### 3.9.3 dict 작동 방식에 의한 영향
#### 키 객체는 반드시 해시 가능해야 한다
다음 요구사항을 모두 만족하는 객체는 해시 가능하다.  
1. 객체의 수명 주기 동안 언제나 동일한 값을 반환하는 `__hash__()`메서드를 제공해서 hash() 함수를 지원한다.
2. `__eq__()`메서드를 통해 동치성을 판단할 수 있다.
3. a == b 가 참이면 hash(a) == hash(b)도 반드시 참이어야 한다.
  
사용자 정의 자료형은 id()를 해시값으로 사용하고 모든 객체는 서로 동일하지 않으므로 기본적으로 해시 가능하다.

사용자 정의 `__eq__()`메서드가 가변 상태에 기반하는 경우 `__hash__()`메서드는 반드시 unhashable type: 'Myclass'같은 형태의 메시지와 함께 TypeError를 발생시켜야 한다.

#### dict의 메모리 오버헤드가 크다
해시 테이블이 제대로 작동하려면 빈 공간이 충분해야 하므로, dict의 메모리 공간 효율성은 높지 않다.  
많은 양의 레코드를 처리하는 경우에는, JSON 형태로 각 레코드에 하나의 dict를 할당해서 딕셔너리의 리스트를 사용하는 것보다 튜플이나 명명된 튜플의 리스트에 저장하는 것이 좋다.  
dict를 튜플로 교체하면, 레코드마다 하나의 해시 테이블을 가져야 하는 부담과 레코드마다 필드명을 다시 저장해야 하는 부담을 제거함으로써 메모리 사용량을 줄일 수 있다.  
사용자 정의 자료형의 경우 `__slots__`클래스 속성을 이용해서 객체 속성 저장소를 dict에-서 튜플로 변경할 수 있다. 

#### 키 검색이 아주 빠르다
dict는 속도를 위해 공간을 포기하는 예다. 메모리 오버헤드가 상당히 크지만, 메모리에 로딩되는 한 딕셔너리 크기와 무관하게 빠른 접근 속도를 제공한다.  

#### 키 순서는 삽입 순서에 따라 달라진다
**Python 3.6 이후 버전부터는 순서가 보장됨**  
해시 충돌이 발생하면 두 번째 키는 충돌이 발생하지 않았을 때의 정상적인 위치와 다른 곳에 놓이게 된다.  
따라서 dict([(key1, value1), (key2, value2)])로 생성한 딕셔너리와 dict([(key2, value2), (key1, value1)])으로 생성한 딕셔너리는 동일하지만, key1과 key2의 해시가 충돌하면 키의 순서는 달라진다.  
생성된 딕셔너리는 순서는 달라도 모두 동일하다고 판단된다.

In [19]:
# 인구 10대 국가의 국제전화 코드
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'),
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan'),
]

d1 = dict(DIAL_CODES)                                  # 1
print('d1:', d1.keys())
d2 = dict(sorted(DIAL_CODES))                          # 2
print('d2:', d2.keys())
d3 = dict(sorted(DIAL_CODES, key=lambda x:x[1]))      # 3
print('d3:', d3.keys())
assert d1 == d2 and d2 == d3                         # 4

d1: dict_keys([86, 91, 1, 62, 55, 92, 880, 234, 7, 81])
d2: dict_keys([1, 7, 55, 62, 81, 86, 91, 92, 234, 880])
d3: dict_keys([880, 55, 86, 91, 62, 81, 234, 92, 7, 1])


1. d1: 인구가 많은 순서대로 정렬된 튜플로 생성
2. d2: 국제전화 코드로 정렬된 튜플로 생성
3. d3: 국가명으로 정렬된 튜플로 생성
4. 딕셔너리가 모두 동일한 키-값 쌍을 갖고 있기 때문에 동일하다고 판단된다.

#### 딕셔너리에 항목을 추가하면 기존 키의 순서가 변경될 수 있다.
dict에 항목을 추가할 때마다 파이썬 인터프리터는 그 딕셔너리의 해시 테이블 크기를 늘릴지 판단한다.  
그리고 더 큰 해시 테이블을 새로 만들어서 기존 항목을 모두 새 테이블에 추가한다.  
이 과정 동안 기존과 다르게 해시 충돌이 발생해서 새로운 해시 테이블에서의 키 순서가 달라질 수 있다.  
이 모든 것은 구현 알고리즘에 따라 달라지므로, 이 현상이 언제 발생할지 정확히 예측할 수 없다.  
그리고 딕셔너리 키를 반복하는 도중에 항목을 변경하는 경우에는 원하는 대로 항목을 검색하지 못하는 경우가 발생할 수 있다.  
심지어 항목을 추가하기 전에 이미 있떤 항목도 제대로 검색하지 못할 수 있다.  
그렇기 때문에 딕셔너리를 반복하는 동안 딕셔너리의 내용을 변경하는 것은 좋지 않은 방법이다.  
딕셔너리를 검색하면서 항목을 추가해야 하는 경우에는 다음 두 단계로 수행한다.  
1. 처음부터 끝까지 딕셔너리를 검색하면서 필요한 항목은 별도의 딕셔너리에 추가한다.
2. 별도의 딕셔너리로 원래 딕셔너리를 갱신한다.  
> 파이썬 3에서 keys().items(), values() 메서드가 반환하는 딕셔너리 뷰는, 파이썬 2에서 이 메서드들이 반환한 리스트와 달리 일종의 집합처럼 동작한다. 이 뷰는 딕셔너리의 내용을 복제하지 않고 동적으로 갱신되므로, dict의 변경사항을 즉시 반영한다.

In [21]:
a = {1:2, 2:3, 3:4}
b = a.keys()
print(b)
a.popitem()
print(b)

dict_keys([1, 2, 3])
dict_keys([1, 2])


### 3.9.4 집합의 작동 방식 - 현실적으로 미치는 영향
**set과 frozenset**도 해시 테이블을 이용해서 구현하지만, 각 **버킷이 항목에 대한 참조만을 담고 있다**는 점이 다르다.  
항목 자체가 dict에서의 키처럼 사용되지만, 이 키를 통해 접근할 값이 없다.  
3.9.3 절에서 설명한 내용이 집합에도 모두 적용되므로, 이미 설명한 내용을 반복할 필요 없이, set에 대한 설명을 다음과 같이 간단히 정리할 수 있다.  
- set 요소는 모두 해시 가능한 객체여야 한다.
- set의 메모리 오버헤드가 상당히 크다.
- 집합에 속해 있는지 매우 효율적으로 검사할 수 있다.
- 요소의 순서는 요소를 추가한 순서에 따라 달라진다.
- 요소를 집합에 추가하면 다른 요소의 순서가 바뀔 수 있다.

## 3.10 요약
딕셔너리는 파이썬의 핵심이다. 기본 dict 외에 표준 라이브러리에서 defaultdict, OrderdDict, ChainMap, Counter 등 바로 사용할 수 있는 간단한 매핑형을 제공한다.  
이 딕셔너리들은 모두 collections 모듈에 정의되어 있다. 또한 확장이 쉬운 UserDict클래스도 제공한다.  
대부분의 매핑형은 setdefault()와 update()라는 강력한 메서드를 제공한다.  
setdefault()메서드는 검색 키가 존재하면 해당 키에 대한 값을 가져오고, 존재하지 않으면 기본값으로 해당 키를 생성한 후 기본값을 반환한다.  
update() 메서드는 다른 매핑형, 키-값 쌍을 제공하는 반복형, 키워드 인수로부터 항목을 가져와서 대량으로 데이터를 추가하거나 덮어쓸 수 있다.  
매핑 생성자도 내부적으로 update()메서드를 사용하므로 매핑형, 반복형, 키워드 인수로부터 객체를 초기화할 수 있다.  
매핑 API에서 제공하는 `__missing__()`메서드는 멋진 연결고리로서, 키를 찾을 수 없을 때 발생하는 일을 정의할 수 있게 해준다.  
collections.abc 모듈은 참조와 자료형 검사를 위해 Mapping과 MutableMapping 추상 베이스 클래스를 제공한다.  
types 모듈에서 제공하는 MappingProxyType 클래스는 잘 알려져 있지 않지만, 불변형 매핑을 생성한다.  
그리고 Set과 MutableSet에 대한 추상 베이스 클래스도 제공한다.  
dict와 set의 기반이 되는 해시 테이블은 상당히 빠르다.  
해시 테이블을 이해하면 항목들의 순서가 정렬되어 잇지 않은 이유 및 심지어 조용히 재정렬된느 이유를 알 수 있다.  
해시 테이블은 속도가 빠른 반면 메로리 공간을 많이 사용한다.